<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Data Wrangling Lab**


Estimated time needed: **45** minutes


In this lab, you will perform data wrangling tasks to prepare raw data for analysis. Data wrangling involves cleaning, transforming, and organizing data into a structured format suitable for analysis. This lab focuses on tasks like identifying inconsistencies, encoding categorical variables, and feature transformation.


## Objectives


After completing this lab, you will be able to:


- Identify and remove inconsistent data entries.

- Encode categorical variables for analysis.

- Handle missing values using multiple imputation strategies.

- Apply feature scaling and transformation techniques.


#### Intsall the required libraries


In [1]:
!pip install pandas
!pip install matplotlib

## Tasks


#### Step 1: Import the necessary module.


### 1. Load the Dataset


<h5>1.1 Import necessary libraries and load the dataset.</h5>


Ensure the dataset is loaded correctly by displaying the first few rows.


In [2]:
# Import necessary libraries
import pandas as pd

# Load the Stack Overflow survey data
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/n01PQ9pSmiRX6520flujwQ/survey-data.csv"
df = pd.read_csv(dataset_url)

# Display the first few rows
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

#### 2. Explore the Dataset


<h5>2.1 Summarize the dataset by displaying the column data types, counts, and missing values.</h5>


In [3]:
# Write your code here
# Display basic info: column names, data types, and non-null counts
print("📋 Dataset Info:")
print(df.info())

# Display number of missing values per column
print("\n❗ Missing Values per Column:")
print(df.isnull().sum())


📋 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65437 entries, 0 to 65436
Columns: 114 entries, ResponseId to JobSat
dtypes: float64(13), int64(1), object(100)
memory usage: 56.9+ MB
None

❗ Missing Values per Column:
ResponseId                 0
MainBranch                 0
Age                        0
Employment                 0
RemoteWork             10631
                       ...  
JobSatPoints_11        35992
SurveyLength            9255
SurveyEase              9199
ConvertedCompYearly    42002
JobSat                 36311
Length: 114, dtype: int64


<h5>2.2 Generate basic statistics for numerical columns.</h5>


In [4]:
# Write your code here
# Generate descriptive statistics for numerical columns
print("📊 Descriptive Statistics for Numerical Columns:")
print(df.describe())


📊 Descriptive Statistics for Numerical Columns:
         ResponseId      CompTotal       WorkExp  JobSatPoints_1  \
count  65437.000000   3.374000e+04  29658.000000    29324.000000   
mean   32719.000000  2.963841e+145     11.466957       18.581094   
std    18890.179119  5.444117e+147      9.168709       25.966221   
min        1.000000   0.000000e+00      0.000000        0.000000   
25%    16360.000000   6.000000e+04      4.000000        0.000000   
50%    32719.000000   1.100000e+05      9.000000       10.000000   
75%    49078.000000   2.500000e+05     16.000000       22.000000   
max    65437.000000  1.000000e+150     50.000000      100.000000   

       JobSatPoints_4  JobSatPoints_5  JobSatPoints_6  JobSatPoints_7  \
count    29393.000000    29411.000000    29450.000000     29448.00000   
mean         7.522140       10.060857       24.343232        22.96522   
std         18.422661       21.833836       27.089360        27.01774   
min          0.000000        0.000000        0.

### 3. Identifying and Removing Inconsistencies


<h5>3.1 Identify inconsistent or irrelevant entries in specific columns (e.g., Country).</h5>


In [5]:
# Write your code here
# Step 1: Check the unique values in the Country column
unique_countries = df['Country'].unique()
print("Unique country entries:\n", unique_countries)

# Step 2: Count how many times each country appears (including strange or inconsistent ones)
country_counts = df['Country'].value_counts(dropna=False)
print("\nCountry counts:\n", country_counts)

# Step 3 (optional): Check for null/blank/irrelevant values
print("\nMissing values in 'Country':", df['Country'].isnull().sum())


Unique country entries:
 ['United States of America'
 'United Kingdom of Great Britain and Northern Ireland' 'Canada' 'Norway'
 'Uzbekistan' 'Serbia' 'Poland' 'Philippines' 'Bulgaria' 'Switzerland'
 'India' 'Germany' 'Ireland' 'Italy' 'Ukraine' 'Australia' 'Brazil'
 'Japan' 'Austria' 'Iran, Islamic Republic of...' 'France' 'Saudi Arabia'
 'Romania' 'Turkey' 'Nepal' 'Algeria' 'Sweden' 'Netherlands' 'Croatia'
 'Pakistan' 'Czech Republic' 'Republic of North Macedonia' 'Finland'
 'Slovakia' 'Russian Federation' 'Greece' 'Israel' 'Belgium' 'Mexico'
 'United Republic of Tanzania' 'Hungary' 'Argentina' 'Portugal'
 'Sri Lanka' 'Latvia' 'China' 'Singapore' 'Lebanon' 'Spain' 'South Africa'
 'Lithuania' 'Viet Nam' 'Dominican Republic' 'Indonesia' 'Kosovo'
 'Morocco' 'Taiwan' 'Georgia' 'San Marino' 'Tunisia' 'Bangladesh'
 'Nigeria' 'Liechtenstein' 'Denmark' 'Ecuador' 'Malaysia' 'Albania'
 'Azerbaijan' 'Chile' 'Ghana' 'Peru' 'Bolivia' 'Egypt' 'Luxembourg'
 'Montenegro' 'Cyprus' 'Paraguay' 'Kazakhst

<h5>3.2 Standardize entries in columns like Country or EdLevel by mapping inconsistent values to a consistent format.</h5>


In [6]:
## Write your code here
# Step 1: Create a mapping dictionary for inconsistent country names
country_mapping = {
    'United States of America': 'USA',
    'United States': 'USA',
    'U.S.': 'USA',
    'US': 'USA',
    'India': 'India',
    'IN': 'India',
    'UK': 'United Kingdom',
    'England': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Deutschland': 'Germany',
    'Россия': 'Russia',
    # Add more mappings as needed
}

# Step 2: Apply the mapping to the Country column
df['Country'] = df['Country'].replace(country_mapping)

# Step 3: Verify the standardization
print("Standardized country entries:\n", df['Country'].value_counts())


# Step 1: Create a mapping for education levels
education_mapping = {
    "Bachelor’s degree (B.A., B.S., B.Eng., etc.)": "Bachelor’s degree",
    "Master’s degree (M.A., M.S., M.Eng., MBA, etc.)": "Master’s degree",
    "Some college/university study without earning a degree": "No degree",
    "Professional degree (JD, MD, etc.)": "Professional degree",
    "I never completed any formal education": "No degree",
    "Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)": "Secondary school",
    # Add more as needed
}

# Step 2: Apply the mapping
df['EdLevel'] = df['EdLevel'].replace(education_mapping)

# Step 3: Verify
print("Standardized EdLevel entries:\n", df['EdLevel'].value_counts())


Standardized country entries:
 Country
USA                                                     11095
Germany                                                  4947
India                                                    4231
United Kingdom of Great Britain and Northern Ireland     3224
Ukraine                                                  2672
                                                        ...  
Micronesia, Federated States of...                          1
Nauru                                                       1
Chad                                                        1
Djibouti                                                    1
Solomon Islands                                             1
Name: count, Length: 185, dtype: int64
Standardized EdLevel entries:
 EdLevel
Bachelor’s degree                                 24942
Master’s degree                                   15557
No degree                                          7651
Secondary school                 

### 4. Encoding Categorical Variables


<h5>4.1 Encode the Employment column using one-hot encoding.</h5>


In [7]:
## Write your code here
# Perform one-hot encoding on the 'Employment' column
employment_encoded = pd.get_dummies(df['Employment'], prefix='Employment')

# Optionally, concatenate the encoded columns back to the original DataFrame
df = pd.concat([df, employment_encoded], axis=1)

# Optionally, drop the original Employment column if no longer needed
# df.drop('Employment', axis=1, inplace=True)

# Display the first few rows to verify
print(df.head())


   ResponseId                      MainBranch                 Age  \
0           1  I am a developer by profession  Under 18 years old   
1           2  I am a developer by profession     35-44 years old   
2           3  I am a developer by profession     45-54 years old   
3           4           I am learning to code     18-24 years old   
4           5  I am a developer by profession     18-24 years old   

            Employment RemoteWork   Check  \
0  Employed, full-time     Remote  Apples   
1  Employed, full-time     Remote  Apples   
2  Employed, full-time     Remote  Apples   
3   Student, full-time        NaN  Apples   
4   Student, full-time        NaN  Apples   

                                    CodingActivities  \
0                                              Hobby   
1  Hobby;Contribute to open-source projects;Other...   
2  Hobby;Contribute to open-source projects;Other...   
3                                                NaN   
4                                 

### 5. Handling Missing Values


<h5>5.1 Identify columns with the highest number of missing values.</h5>


In [8]:
## Write your code here
# Count missing values in each column
missing_counts = df.isnull().sum()

# Sort the columns by missing values in descending order
missing_sorted = missing_counts.sort_values(ascending=False)

# Display the top columns with the most missing values
print("Columns with the highest number of missing values:")
print(missing_sorted.head(10))  # change 10 to any number if you want more/fewer columns


Columns with the highest number of missing values:
AINextMuch less integrated       64289
AINextLess integrated            63082
AINextNo change                  52939
AINextMuch more integrated       51999
EmbeddedAdmired                  48704
EmbeddedWantToWorkWith           47837
EmbeddedHaveWorkedWith           43223
ConvertedCompYearly              42002
AIToolNot interested in Using    41023
AINextMore integrated            41009
dtype: int64


<h5>5.2 Impute missing values in numerical columns (e.g., `ConvertedCompYearly`) with the mean or median.</h5>


In [11]:
## Write your code here
# Check for missing values in ConvertedCompYearly
missing_before = df['ConvertedCompYearly'].isnull().sum()
print(f"Missing values before imputation: {missing_before}")

# Impute missing values with the mean
mean_value = df['ConvertedCompYearly'].mean()
df['ConvertedCompYearly'].fillna(mean_value, inplace=True)

# Verify that the missing values are filled
missing_after = df['ConvertedCompYearly'].isnull().sum()
print(f"Missing values after imputation: {missing_after}")




Missing values before imputation: 0
Missing values after imputation: 0


/tmp/ipykernel_1521/2508434013.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConvertedCompYearly'].fillna(mean_value, inplace=True)


<h5>5.3 Impute missing values in categorical columns (e.g., `RemoteWork`) with the most frequent value.</h5>


In [12]:
## Write your code here
# Check for missing values before imputation
missing_before = df['RemoteWork'].isnull().sum()
print(f"Missing values in 'RemoteWork' before imputation: {missing_before}")

# Find the most frequent value in the column
most_frequent = df['RemoteWork'].mode()[0]

# Impute missing values
df['RemoteWork'].fillna(most_frequent, inplace=True)

# Check again after imputation
missing_after = df['RemoteWork'].isnull().sum()
print(f"Missing values in 'RemoteWork' after imputation: {missing_after}")


Missing values in 'RemoteWork' before imputation: 10631
Missing values in 'RemoteWork' after imputation: 0


/tmp/ipykernel_1521/3621034829.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['RemoteWork'].fillna(most_frequent, inplace=True)


### 6. Feature Scaling and Transformation


<h5>6.1 Apply Min-Max Scaling to normalize the `ConvertedCompYearly` column.</h5>


In [13]:
## Write your code here
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Ensure there are no missing values in the compensation column before scaling
df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')

# Drop rows with missing compensation data for scaling
df_scaling = df[['ConvertedCompYearly']].dropna().copy()

# Initialize the scaler
scaler = MinMaxScaler()

# Apply Min-Max Scaling
df_scaling['ConvertedCompYearly_MinMax'] = scaler.fit_transform(df_scaling[['ConvertedCompYearly']])

# Merge the scaled values back into the original DataFrame using index
df = df.merge(df_scaling[['ConvertedCompYearly_MinMax']], left_index=True, right_index=True, how='left')

# Display summary statistics of the scaled column
print(df['ConvertedCompYearly_MinMax'].describe())


count    65437.000000
mean         0.005300
std          0.006875
min          0.000000
25%          0.005300
50%          0.005300
75%          0.005300
max          1.000000
Name: ConvertedCompYearly_MinMax, dtype: float64


<h5>6.2 Log-transform the ConvertedCompYearly column to reduce skewness.</h5>


In [14]:
## Write your code here
import numpy as np

# Ensure ConvertedCompYearly is numeric
df['ConvertedCompYearly'] = pd.to_numeric(df['ConvertedCompYearly'], errors='coerce')

# Apply log1p transformation (handles 0 and positive values)
df['ConvertedCompYearly_Log'] = np.log1p(df['ConvertedCompYearly'])

# Check the transformation
print(df['ConvertedCompYearly_Log'].describe())


count    65437.000000
mean        11.156906
std          0.884085
min          0.693147
25%         11.363918
50%         11.363918
75%         11.363918
max         16.604010
Name: ConvertedCompYearly_Log, dtype: float64


### 7. Feature Engineering


<h5>7.1 Create a new column `ExperienceLevel` based on the `YearsCodePro` column:</h5>


In [15]:
## Write your code here
# Handle special cases (e.g., 'Less than 1 year', 'More than 50 years')
df['YearsCodePro'] = df['YearsCodePro'].replace({
    'Less than 1 year': '0',
    'More than 50 years': '51'
})

# Convert to numeric, setting errors='coerce' to handle non-numeric values
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')

# Define ExperienceLevel based on ranges
def categorize_experience(years):
    if pd.isna(years):
        return 'Unknown'
    elif years <= 2:
        return 'Beginner'
    elif years <= 5:
        return 'Intermediate'
    else:
        return 'Expert'

# Apply function to create new column
df['ExperienceLevel'] = df['YearsCodePro'].apply(categorize_experience)

# Check results
print(df[['YearsCodePro', 'ExperienceLevel']].head())


   YearsCodePro ExperienceLevel
0           NaN         Unknown
1          17.0          Expert
2          27.0          Expert
3           NaN         Unknown
4           NaN         Unknown


### Summary


In this lab, you:

- Explored the dataset to identify inconsistencies and missing values.

- Encoded categorical variables for analysis.

- Handled missing values using imputation techniques.

- Normalized and transformed numerical data to prepare it for analysis.

- Engineered a new feature to enhance data interpretation.


Copyright © IBM Corporation. All rights reserved.
